# Krankenversicherung / Aufgabenserie vom 01.11.2024

## 1. Rückstellungsberechnung mit Chain-Ladder

Gegeben sei das folgende Abwicklungsdreieck mit den im Behandlungsmonat i angefallenen und im Abrechnugnsmonat k bezahlten Leistungen $S_{i,k}$:

|                   | Abrechnungs-monat k |        |        |        |
| ----------------- | ------------------- | ------ | ------ | ------ |
| Behandlungs-monat | 1                   | 2      | 3      | 4      |
| 1                 | 58000               | 276100 | 111700 | 20400  |
| 2                 |                     | 100900 | 480319 | 194319 |
| 3                 |                     |        | 120800 | 575050 |
| 4                 |                     |        |        | 130900 |

a) Berechnen Sie den Rückstellungsbedarf mit Hilfe des Chain-Ladder Verfahrens. 

b) Angenommen in den darauffolgenden Behandlungsmonaten 5 und 6 fallen Leistungen von total 1'000'000 (Behandlungsmonat 5) bzw. 1'100'000 (Behandlungsmonat 6) an. Das Abwicklungsmuster bleibe dasselbe wie in den vorangehenden Monaten 1 bis 4. Wie viele Leistungen werden in den Abrechnungsmonaten 5 und 6 abgerechnet? 

c) Schildern Sie in ein paar wenigen Stichworten, wie sich Chain-Ladder nicht bloss als Algorithmus sondern auch als stochastisches Modell implementieren lässt. 

In [1]:
# Daten laden
library(tidyverse)

tria_beh_mon <- c(
	c(58000, 276100, 111700, 20400),
	c(0, 100900, 480319, 194319),
	c(0, 0, 120800, 575050),
	c(0, 0, 0, 130900)
) |>
	matrix(nrow = 4) |>
	t()

tria_beh_mon

Warning message:
"Paket 'tidyverse' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'ggplot2' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'tibble' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'tidyr' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'readr' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'purrr' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'dplyr' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'stringr' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'forcats' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'lubridate' wurde unter R Version 4.2.3 erstellt"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────

58000,276100,111700,20400
0,100900,480319,194319
0,0,120800,575050
0,0,0,130900


In [2]:
# In timelag-Ansicht wechseln
tria_rel <- tria_beh_mon
for (i in 1:nrow(tria_rel)) tria_rel[i,] <- lead(tria_rel[i,], i - 1)
tria_rel

58000,276100,111700,20400
100900,480319,194319,NA
120800,575050,NA,NA
130900,NA,NA,NA


In [3]:
# Kumulierte Ansicht
tria_kum <- tria_rel
for (j in 2:ncol(tria_kum)) tria_kum[,j] <- tria_kum[,j] + tria_kum[,j - 1]
tria_kum

58000,334100,445800,466200
100900,581219,775538,NA
120800,695850,NA,NA
130900,NA,NA,NA


In [4]:
# Faktoren berechnen
tria_kum_prev <- tria_kum
tria_kum_prev[!is.na(tria_kum_prev)] <- 1
for (i in 1:nrow(tria_kum_prev)) tria_kum_prev[i,] <- lead(tria_kum_prev[i,], 1)
tria_kum_prev <- tria_kum_prev * tria_kum

factors <- c()
for (j in 2:ncol(tria_kum)) {
	sum_prev <- sum(tria_kum_prev[,j - 1], na.rm = TRUE)
	sum_base <- sum(tria_kum[,j], na.rm = TRUE)
	factors <- append(factors, sum_base / sum_prev)
}
factors

[1] 5.760347 1.334330 1.045760

In [5]:
# Abrechnungsquadrat rechnen
quadr_kum <- tria_kum
for (j in 2:ncol(quadr_kum)) quadr_kum[,j] <- if_else(is.na(quadr_kum[,j]), factors[j - 1] * quadr_kum[,j - 1], quadr_kum[,j])
quadr_kum

58000,334100.0,445800.0,466200.0
100900,581219.0,775538.0,811027.0
120800,695850.0,928493.8,970982.1
130900,754029.4,1006124.4,1052165.1


In [6]:
# nicht kumulierte Darstellung
quadr <- quadr_kum
for (j in 2:ncol(quadr)) quadr[,j] <- quadr_kum[,j] - quadr_kum[,j - 1]
quadr

58000,276100.0,111700.0,20400.00
100900,480319.0,194319.0,35488.95
120800,575050.0,232643.8,42488.28
130900,623129.4,252095.0,46040.68


In [7]:
# Ansicht nach abrechnungsmonaten
abr_mon <- cbind(quadr, matrix(NA_real_,4,3))
for (i in 1:nrow(abr_mon)) abr_mon[i,] <- lag(abr_mon[i,], i - 1)
abr_mon

58000,276100,111700,20400,NA,NA,NA
NA,100900,480319,194319,35488.95,NA,NA
NA,NA,120800,575050,232643.83,42488.28,NA
NA,NA,NA,130900,623129.40,252094.98,46040.68


In [9]:
# Summen über Abrechnugnsmonate
sum_abr_mon <- summarise(abr_mon |>  as_tibble(), across(everything(), ~ sum(.x, na.rm = TRUE)))
sum_lst <- pivot_longer(sum_abr_mon, everything(), names_to = "abrechnungsmonat", values_to = "betrag", names_prefix = "V")
sum_lst
sum_lst |> filter(abrechnungsmonat > 4) |> summarise(sum(betrag))

abrechnungsmonat,betrag
<chr>,<dbl>
1,58000.00
2,377000.00
3,712819.00
4,920669.00
5,891262.18
6,294583.25
7,46040.68


sum(betrag)
<dbl>
1231886
